In [1]:
import sys 
sys.path.append('../')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from NN import network, activation_functions, loss_functions
from NN import ANN
X = np.array([[1,2],[3,4]])


## First Code running

In [2]:
# Random Init
teste = ANN.ANN("sigmoid")
teste.initialize_random_weights(2, [3], 2)
teste.show_weights()
teste.show_setup()
Y, aY = teste.foward_propagation(X)
print("Y\n",Y)
print("\na(Y)\n",aY)

Hidden Layer  1
[[0.38756401 0.31665851 0.42402114]
 [0.22774475 0.9877569  0.76386569]
 [0.93595815 0.27251824 0.86651682]] 

Output Layer 
[[0.21668265 0.06748016]
 [0.96205045 0.04683531]
 [0.40212244 0.00591343]
 [0.06774415 0.40054822]] 

 Input size:  2
 Number of hidden layers:  1
 Number of perceptrons at each layer: 
 HL 1: 3
 Number of classes: 2 

Y
 [[1.51662691 0.49589861]
 [1.63569651 0.51950025]]

a(Y)
 [[0.82004124 0.62149501]
 [0.83694851 0.6270309 ]]


## Comparing with the new Implementation by using the same weights

In [3]:
from importlib import reload
reload(network)
model = network.NN(loss='mse')
model.add_layer(network.Layer(2, 3, 'sigmoid', weights=teste.hidden_layers[0], label="H1"))
model.add_layer(network.Layer(3, 2, 'sigmoid', weights=teste.output_layer, label="out"))
model.show_weights()

Y, aY = model.feed_forward(X)

print("Y\n",Y)
print("\na(Y)\n",aY)



Model Weights
-------------------------------
H1      (input=2, neurons=3, activation=sigmoid)
[[0.38756401 0.31665851 0.42402114]
 [0.22774475 0.9877569  0.76386569]
 [0.93595815 0.27251824 0.86651682]]
out     (input=3, neurons=2, activation=sigmoid)
[[0.21668265 0.06748016]
 [0.96205045 0.04683531]
 [0.40212244 0.00591343]
 [0.06774415 0.40054822]]
-------------------------------
Forwardign H1 weights (2, 2) (3, 3)
Forwardign out weights (2, 3) (4, 2)
Y
 [[1.51662691 0.49589861]
 [1.63569651 0.51950025]]

a(Y)
 [[0.82004124 0.62149501]
 [0.83694851 0.6270309 ]]


## Random initialization

In [4]:
from importlib import reload
reload(network)
model = network.NN(loss='mse')
model.add_layer(network.Layer(2, 3, 'sigmoid',  label="H1"))
model.add_layer(network.Layer(3, 2, 'sigmoid',  label="out"))
model.show_weights()

Y, aY = model.feed_forward(X)

print("Y\n",Y)
print("\na(Y)\n",aY)


Model Weights
-------------------------------
H1      (input=2, neurons=3, activation=sigmoid)
[[0.06066682 0.96583963 0.5479115 ]
 [0.41136338 0.65547145 0.0899457 ]
 [0.91094766 0.32790101 0.47666563]]
out     (input=3, neurons=2, activation=sigmoid)
[[0.22150463 0.65530396]
 [0.6802047  0.39582851]
 [0.95746223 0.8118135 ]
 [0.50774691 0.14151367]]
-------------------------------
Forwardign H1 weights (2, 2) (3, 3)
Forwardign out weights (2, 3) (4, 2)
Y
 [[2.12959515 1.86871636]
 [2.31728139 1.98144741]]

a(Y)
 [[0.89374657 0.86630968]
 [0.9102982  0.87883537]]


## Softmax Sample and Test

In [5]:
reload(activation_functions)
reload(network)
layer = network.Layer(4, 4, 'softmax', label="output")
val = np.array([[5.1, 3.2, -1.7, -2.0], [5.1, 3.2, -1.7, -2.0]])

layer.feed_forward(val)

Forwardign output weights (2, 4) (5, 4)


(array([[ 5.1,  3.2, -1.7, -2. ],
        [ 5.1,  3.2, -1.7, -2. ]]),
 array([[8.68426824e-01, 1.29889401e-01, 9.67232214e-04, 7.16543248e-04],
        [8.68426824e-01, 1.29889401e-01, 9.67232214e-04, 7.16543248e-04]]))

In [6]:
from importlib import reload
reload(network)
model = network.NN(loss='cross_entropy')
model.add_layer(network.Layer(2, 3, 'sigmoid',  label="H1"))
model.add_layer(network.Layer(3, 2, 'sigmoid',  label="output"))
model.add_layer(network.Layer(2, 2, 'softmax',  label="softmax"))
model.summary()

Y, aY = model.feed_forward(X)

print("Y\n",Y)
print("\na(Y)\n",aY)

Model Summary
-------------------------------
H1      (input=2, neurons=3, activation=sigmoid)
output  (input=3, neurons=2, activation=sigmoid)
softmax (input=2, neurons=2, activation=softmax)
-------------------------------
Forwardign H1 weights (2, 2) (3, 3)
Forwardign output weights (2, 3) (4, 2)
Forwardign softmax weights (2, 2) (3, 2)
Y
 [[0.86858984 0.95729205]
 [0.88366535 0.96444314]]

a(Y)
 [[0.47783898 0.52216102]
 [0.47981653 0.52018347]]


## Testing the loss functions 

### Mean Squared Error

In [7]:
reload(loss_functions)
I = np.array([[1.5], [4.5], [9]])
Y_  = np.array([[0.5], [1.5], [3]])
W = np.array([[1.8]])
lr = 0.005
eps  = 0.0001
max_it = 50
for i in range(max_it):
    aY  = I * W
    print(i, aY.flatten())
    if np.absolute((Y_ - aY)).mean() < eps: 
        break
    dC = loss_functions.mse_derivative(I, aY, Y_)
    W = W - lr * dC
    
    


0 [ 2.7  8.1 16.2]
1 [ 2.59  7.77 15.54]
2 [ 2.4855  7.4565 14.913 ]
3 [ 2.386225  7.158675 14.31735 ]
4 [ 2.29191375  6.87574125 13.7514825 ]
5 [ 2.20231806  6.60695419 13.21390837]
6 [ 2.11720216  6.35160648 12.70321296]
7 [ 2.03634205  6.10902615 12.21805231]
8 [ 1.95952495  5.87857485 11.75714969]
9 [ 1.8865487   5.6596461  11.31929221]
10 [ 1.81722127  5.4516638  10.9033276 ]
11 [ 1.7513602   5.25408061 10.50816122]
12 [ 1.68879219  5.06637658 10.13275316]
13 [1.62935258 4.88805775 9.7761155 ]
14 [1.57288495 4.71865486 9.43730972]
15 [1.51924071 4.55772212 9.11544424]
16 [1.46827867 4.40483601 8.80967203]
17 [1.41986474 4.25959421 8.51918842]
18 [1.3738715 4.1216145 8.243229 ]
19 [1.33017793 3.99053378 7.98106755]
20 [1.28866903 3.86600709 7.73201418]
21 [1.24923558 3.74770673 7.49541347]
22 [1.2117738  3.6353214  7.27064279]
23 [1.17618511 3.52855533 7.05711065]
24 [1.14237585 3.42712756 6.85425512]
25 [1.11025706 3.33077118 6.66154237]
26 [1.07974421 3.23923262 6.47846525]
27 [1

### Cross Entropy 

In [8]:
reload(loss_functions)
I = np.array([[1.5], [4.5], [9]])
Y_  = np.array([[0.5], [1.5], [3]])
W = np.array([[1.8]])

for i in range(max_it):
    aY  = I * W
    print(i, aY.flatten())
    if np.absolute((Y_ - aY)).mean() < eps: 
        break
    dC = loss_functions.cross_entropy_derivative(I, aY, Y_)
    W = W - lr * dC



0 [ 2.7  8.1 16.2]
1 [1.5615 4.6845 9.369 ]
2 [1.01217375 3.03652125 6.0730425 ]
3 [0.74712383 2.2413715  4.48274301]
4 [0.61923725 1.85771175 3.7154235 ]
5 [0.55753197 1.67259592 3.34519184]
6 [0.52775918 1.58327753 3.16655506]
7 [0.5133938  1.54018141 3.08036282]
8 [0.50646251 1.51938753 3.03877506]
9 [0.50311816 1.50935448 3.01870897]
10 [0.50150451 1.50451354 3.00902708]
11 [0.50072593 1.50217778 3.00435556]
12 [0.50035026 1.50105078 3.00210156]
13 [0.500169 1.500507 3.001014]
14 [0.50008154 1.50024463 3.00048926]
15 [0.50003934 1.50011803 3.00023607]
16 [0.50001898 1.50005695 3.0001139 ]


### Sum of Squared Errors 

In [9]:
reload(loss_functions)
I = np.array([[1.5], [4.5], [9]])
Y_  = np.array([[0.5], [1.5], [3]])
W = np.array([[1.8]])

for i in range(max_it):
    aY  = I * W
    print(i, aY.flatten())
    if np.absolute((Y_ - aY)).mean() < eps: 
        break
    dC = loss_functions.smd_derivative(I, aY, Y_)
    W = W - lr * dC

0 [ 2.7  8.1 16.2]
1 [1.5615 4.6845 9.369 ]
2 [1.01217375 3.03652125 6.0730425 ]
3 [0.74712383 2.2413715  4.48274301]
4 [0.61923725 1.85771175 3.7154235 ]
5 [0.55753197 1.67259592 3.34519184]
6 [0.52775918 1.58327753 3.16655506]
7 [0.5133938  1.54018141 3.08036282]
8 [0.50646251 1.51938753 3.03877506]
9 [0.50311816 1.50935448 3.01870897]
10 [0.50150451 1.50451354 3.00902708]
11 [0.50072593 1.50217778 3.00435556]
12 [0.50035026 1.50105078 3.00210156]
13 [0.500169 1.500507 3.001014]
14 [0.50008154 1.50024463 3.00048926]
15 [0.50003934 1.50011803 3.00023607]
16 [0.50001898 1.50005695 3.0001139 ]
